# Определим наш датасет
Наш (воображаемый) датасет составлен из **1000 образцов данных** (data samples).

* Каждый сэмпл имеет 100 признаков/свойств/параметров/features.
* Параметр представляет собой нецелое число от 0 до 1
* Сэмплы делятся на 10 категорий. Задачей нейросети будет определениеверной категории для данного сэмпла.
* Мы будем использовать 800 сэмплов для **тренировки** и 200 сэмплов для **аттестации** (проверки).
* Будеим использовать **размер пакета** (batch size) равный 10 для тренировки и аттестации.

In [1]:
import mxnet as mx
import numpy as np
import logging


#составление лога
logging.basicConfig(level=logging.INFO)

sample_count = 1000
train_count = 800
valid_count = sample_count - train_count

feature_count = 100
category_count = 10
batch_size = 10

# Генерация датасета
Будем использовать равномерное распределение для генерации 1000 сэмплов. Они хранятся в NDArray 'X': **1000 строк, 100 колонок.**

In [2]:
X = mx.nd.uniform(low=0, high=1, shape=(sample_count, feature_count))

print(X.shape)
print(X.asnumpy())

(1000, 100)
[[5.48813522e-01 5.92844605e-01 7.15189338e-01 ... 6.07830644e-01
  4.17021990e-01 9.97184813e-01]
 [7.20324516e-01 9.32557344e-01 1.14381080e-04 ... 1.85082078e-01
  2.59262286e-02 9.31540847e-01]
 [5.49662471e-01 9.47730601e-01 4.35322404e-01 ... 8.39949071e-01
  2.90904731e-01 1.21328577e-01]
 ...
 [8.18738341e-02 2.44539641e-02 8.53825390e-01 ... 6.76115692e-01
  1.75401583e-01 9.24937129e-01]
 [3.79189700e-01 8.49168241e-01 7.75433898e-01 ... 4.59736548e-02
  5.40907323e-01 8.03755522e-01]
 [5.59466481e-01 9.35443759e-01 5.55880964e-01 ... 1.00363605e-01
  9.17030871e-01 4.12768424e-01]]


Категориями для этих сэмпллов будут являтся числами от 0 до 9, рандомно сгенерированных и хранящихся в 'Y'.

In [3]:
Y = mx.nd.empty((sample_count,))
for i in range(sample_count):
    Y[i] = np.random.randint(0, category_count)
    
print(Y.shape)
print(Y[0:10].asnumpy())

(1000,)
[9. 9. 1. 8. 8. 4. 9. 6. 7. 9.]


# Разделение набора данных
Дальше мы разделим наш датасет в соотношении 80/20 для тренировки и валидации. Воспользуемся ```NDArray.crop``` функцией для этого. Тут датасет полностью случаен, поэтому мы можем использовать первые 80% для тренировки, а оставшиеся 20% для валидации. На реальном примере нам бы пришлось сначала **перемешать** данные, чтобы избежать возможных совпадений на последовательно-сгенерированных данных.

In [4]:
X_train = mx.nd.crop(X, begin=(0,0), end=(train_count, feature_count))
X_valid = mx.nd.crop(X, begin=(train_count,0), end=(sample_count, feature_count))

Y_train = Y[ : train_count]
Y_valid = Y[train_count : sample_count]

Наши данные готовы.

# Построение сети
Наша сеть очень простая. Посмотрим на каждый слой:
* **input layer** представлен Символом 'data'. Мы свяжем его непосредственно с входными данными позже.

In [5]:
data = mx.sym.Variable('data')

*fc1* (fully-connected) - **первый hidden layer** построен из 64 полностью-связных нейронов, другими словами, каждый параметр во входном слое будет связан с 64 нейронами. Как вы можете видеть, мы используем высокоуровневую ```Symbol.FullyConnected``` функцию, которая более удобна, чем прописывание каждого соединения вручную.

In [6]:
fc1 = mx.sym.FullyConnected(data, name='fully-connected1', num_hidden=64)

Каждый вывод fc1 проходит через [функцию активации ](https://en.wikipedia.org/wiki/Activation_function). Тут мы используем [rectified linear unit](https://en.wikipedia.org/wiki/Rectifier_%28neural_networks%29) или 'relu'.

In [7]:
relu1 = mx.sym.Activation(fc1, name='relu1', act_type="relu")

*fc2* - второй скрытый слой, построенный из 10 полностью-связных нейронов, которые отображают наши 10 категорий. Каждый нейрон выводит дробное число, отмасштабированное произвольно. Наибольшее из 10 значений отобразит самую вероятную категорию сэмпла.

In [8]:
fc2 = mx.sym.FullyConnected(relu1, name='fully-connected2', num_hidden=category_count)

**Результирующий слой** применит [Softmax](https://ru.wikipedia.org/wiki/Softmax) функцию к 10 значениям с fc2 слоя: они превратятся в 10 значений между 0 и 1, которые в сумме дадут 1. Каждое значение отразит **предсказанную вероятность для каждой категории**, наибольшее укажет на самую вероятную.

In [9]:
out = mx.sym.SoftmaxOutput(fc2, name='softmax')
mod = mx.mod.Module(out)

# Создание Итератора
Нейросеть не тренеруется на одном сэмле в единицу времени, это неэффективно с точки зрения производительности. Вместо этого мы используем **пакеты** (batches), фиксированное число сэмплов, по-другому.

Чтобы доставить эти пакеты сети, мы вынуждены построить итератор, используя ```NDArrayIter``` функцию. Это параметры наших данных для тренировки, категории (MXNet называет их **labels** (названия)) и размер пакета.

Как вы видите, мы можем итерироваться по датасету, 10 сэмплов и 10 названий в единицу времени. Потом вызовем функцию ```reset()``` для восстановления итератора в начальное состояние.

In [10]:
train_iter = mx.io.NDArrayIter(data=X_train, label=Y_train, batch_size=batch_size)

for batch in train_iter: #для пакета в итераторе
    print(batch.data)
    print(batch.label)
    break

train_iter.reset()

[
[[5.48813522e-01 5.92844605e-01 7.15189338e-01 8.44265759e-01
  6.02763355e-01 8.57945621e-01 5.44883192e-01 8.47251713e-01
  4.23654795e-01 6.23563707e-01 6.45894110e-01 3.84381711e-01
  4.37587202e-01 2.97534615e-01 8.91772985e-01 5.67129776e-02
  9.63662744e-01 2.72656292e-01 3.83441508e-01 4.77665126e-01
  7.91725039e-01 8.12168717e-01 5.28894901e-01 4.79977161e-01
  5.68044543e-01 3.92784804e-01 9.25596654e-01 8.36078763e-01
  7.10360557e-02 3.37396175e-01 8.71292949e-02 6.48171902e-01
  2.02183984e-02 3.68241549e-01 8.32619846e-01 9.57155168e-01
  7.78156757e-01 1.40350774e-01 8.70012164e-01 8.70087266e-01
  9.78618324e-01 4.73608047e-01 7.99158573e-01 8.00910771e-01
  4.61479366e-01 5.20477474e-01 7.80529201e-01 6.78879559e-01
  1.18274420e-01 7.20632672e-01 6.39921010e-01 5.82019806e-01
  1.43353283e-01 5.37373245e-01 9.44668889e-01 7.58615613e-01
  5.21848321e-01 1.05907604e-01 4.14661944e-01 4.73600417e-01
  2.64555603e-01 1.86332345e-01 7.74233699e-01 7.36918151e-01
  4.56

Сеть готова учиться.
# Тренируем модель
Сперва свяжем входящий символ с датасетом (сэмплы и названия). Тут и нужен итератор

In [11]:
mod.bind(data_shapes=train_iter.provide_data, label_shapes=train_iter.provide_label)

Дальше инициализируем веса нейронов. Это очень важный шаг: инициализирующий их "правильным" способ поможет сети учиться намного быстрее. Инициализатор Завьера (Xavier) - один из способов.

In [12]:
#Допустимо, но неэффективно
mod.init_params()
#Много лучше
mod.init_params(initializer=mx.init.Xavier(magnitude=2.))

/home/gleb/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  after removing the cwd from sys.path.


Теперь нужно определить параметры оптимизации:
* будем использовать [Стохастический градиентный спуск](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) (Stochastic gradient descent или SGD), который долго использовался в ML и DL приложениях.
* установим **learning rate** равный 0.1, типичное значение для SGD.

In [13]:
mod.init_optimizer(optimizer='sgd', optimizer_params=(('learning_rate', 0.1), ))

Теперь можно тренировать сеть. Проведём 100 эпох, что значит - все данные 100 раз пройдут через сеть (в пакетах по 10).

In [14]:
mod.fit(train_iter, num_epoch=100)

/home/gleb/anaconda3/lib/python3.6/site-packages/mxnet/module/base_module.py:464: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  allow_missing=allow_missing, force_init=force_init)
INFO:root:Epoch[0] Train-accuracy=0.091250
INFO:root:Epoch[0] Time cost=0.271
INFO:root:Epoch[1] Train-accuracy=0.096250
INFO:root:Epoch[1] Time cost=0.065
INFO:root:Epoch[2] Train-accuracy=0.097500
INFO:root:Epoch[2] Time cost=0.076
INFO:root:Epoch[3] Train-accuracy=0.090000
INFO:root:Epoch[3] Time cost=0.078
INFO:root:Epoch[4] Train-accuracy=0.091250
INFO:root:Epoch[4] Time cost=0.076
INFO:root:Epoch[5] Train-accuracy=0.090000
INFO:root:Epoch[5] Time cost=0.079
INFO:root:Epoch[6] Train-accuracy=0.090000
INFO:root:Epoch[6] Time cost=0.068
INFO:root:Epoch[7] Train-accuracy=0.090000
INFO:root:Epoch[7] Time cost=0.063
INFO:root:Epoch[8] Train-accuracy=0.092500
INFO:root:Epoch[8] Time cost=0.084
INFO:root:Epoch[9] Train-accuracy=0.103750
INFO:root:Epoch[9] Time cos

INFO:root:Epoch[99] Train-accuracy=1.000000
INFO:root:Epoch[99] Time cost=0.085


Как видно, на [83-84] сеть достигла точности 1.0

Проверим её на наборе для аттестации.

# Аттестация модели
Теперь мы пустим новые сэмплы через сеть, те 20%, что не использовались для обучения.

Сначала создадим итератор. В этот раз используем сэмплы и названия для валидации.

In [15]:
pred_iter = mx.io.NDArrayIter(data=X_valid, label=Y_valid, batch_size=batch_size)

Затем используем ```Module.iter_predict()``` функцию. Как только мы это сделаем, мы сравним **предсказанные названия** с **действительными**. Будем отслеживать точность аттестации, другими словами, как хорошо сеть справляется с набором для валидации.

In [16]:
pred_count = valid_count #всего 200 ответов
correct_preds = total_correct_preds = 0

for preds, i_batch, batch in mod.iter_predict(pred_iter):
    label = batch.label[0].asnumpy().astype(int)
    pred_label = preds[0].asnumpy().argmax(axis=1)
    correct_preds = np.sum(pred_label==label)
    total_correct_preds += correct_preds
    
print('Точность валидации: {0}'.format(1.0 * total_correct_preds / pred_count))

Точность валидации: 0.09


Что тут происходит

```iter_predict()``` возвращает:
* preds - массив NDArrays. Тут он содержит один NDArray, хранящий прдесказанные названия для текущего пакета: для каждого сэмпла у нас есть предсказанные вероятности для всех 10 категорий (10х10 матрица). Итак, мы используем ```argmax()```, чтобы найти индекс наибольшего значения. pred_label это 10-элементный массив, хранящий предсказанные категории для каждого дата сэмла в текущем пакете.
* i_batch - номер пакета
* batch - массив NDArrays. Также содержит один NDArray для текущего пакета. Мы используем его, чтобы найти названия 10 сэмплов в пакете. Храним их в ```label``` numpy массиве (на 10 элементов).

Потом, мы сравниваем номера равных значений в ```labels``` и ```pred_labels```, используя ```Numpy.sum()```.

Наконец, считаем и выводим точность.

Но почему 0.135%? Это действительно плохо. Если нужно было подтверждение, что данные получены случайно, то вот оно.

Суть в том, что мы действительно можем обучить нейросеть чему угодно, но если данные бессмысленны (как у нас), то ничего предсказать и не выйдет. Мусор на вход - мусор на выход.